# Spam Filter

In dit notebook maken we een spamfilter.

## Verover de data

We maken gebruik van publieke data van Apache Spamassassin. Deze data gaan we downloaden van het internet. Zorg dat hieronder bij `SPAM_PATH` de naam van de map staat waar je het resultaat wilt opslaan.

In [5]:
import os
import tarfile
from six.moves import urllib

import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

DOWNLOAD_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"
HAM_URL = DOWNLOAD_ROOT + "20030228_easy_ham.tar.bz2"
SPAM_URL = DOWNLOAD_ROOT + "20030228_spam.tar.bz2"

# verander onderstaande url in de map waarin je het resultaat wilt opslaan
SPAM_PATH = ""

In [6]:
print("dafdaf")

dafdaf


We maken een pythonfunctie om de bestanden van internet te halen. Zoek uit wat deze functie doet.

In [7]:
def fetch_spam_data(spam_url=SPAM_URL, spam_path=SPAM_PATH):
    if not os.path.isdir(spam_path):# als de map nog niet bestaat...
        os.makedirs(spam_path)
    
    for filename, url in (("ham.tar.bz2", HAM_URL), ("spam.tar.bz2", SPAM_URL)):
        path = os.path.join(spam_path, filename)
        
        if not os.path.isfile(path): # als het bestand nog niet bestaat...
            urllib.request.urlretrieve(url, path)
        
        tar_bz2_file = tarfile.open(path)
        tar_bz2_file.extractall(path=SPAM_PATH)
        tar_bz2_file.close()

Na het aanroepen van de functie moet er een map `easy_ham` en een map `spam` zijn toegevoegd binnen `SPAM_PATH`. Roep de functie aan en controleer dat dit het geval is.

In [8]:
fetch_spam_data()

FileNotFoundError: [WinError 3] The system cannot find the path specified: ''

Om makkelijk bij de bestanden te kunnen, maken we voor beide categorieën een lijst met bestandsnamen.

De methode `os.listdir()` geeft je een lijst van bestandsnamen in een map. Die kunnen we eventueel sorteren met `sorted()`. We gebruiken deze functies om zowel voor ham als voor spam een lijst te maken van alle bestandsnamen waarvan de lengte groter is dan 20 tekens.

In [ ]:
HAM_DIR = os.path.join(SPAM_PATH, "easy_ham")
SPAM_DIR = os.path.join(SPAM_PATH, "spam")

ham_filenames = [name for name in sorted(os.listdir(HAM_DIR)) if len(name) > 20]
spam_filenames = [name for name in sorted(os.listdir(SPAM_DIR)) if len(name) > 20]

Hoeveel bestanden met spam zijn er?

Hoeveel bestanden zonder spam (met ham) zijn er?

Laten we eens zo'n bestand bekijken, bijvoorbeeld het 4e bestand zonder spam.

In [ ]:
with open(os.path.join(SPAM_PATH, 'easy_ham', ham_filenames[3])) as myfile:
    lines = myfile.readlines()
print(lines)

Hierboven zie je de ruwe tekst van de e-mail. We kunnen dit iets leesbaarder maken door te zorgen dat elke nieuwe regel in het bestand ook op een eigen regel wordt afgedrukt. Bekijk vervolgens de tekst. Wat valt je op?

In [ ]:
with open(os.path.join(SPAM_PATH, 'easy_ham', ham_filenames[3])) as myfile:
    for line in myfile.readlines():
        print(line)

Wat we zien, is een e-mail zoals een e-mailprogramma dit ontvangt. Een e-mail bestaat uit een header, waar aan we o.a. kunnen zien wie de afzender en ontvanger zijn en langs welke tussenstations de mail verstuurd is. Na de header volgt een lege regel en dan de inhoud van de e-mail.

We kunnen de emails inlezen als tekstbestanden zoals we zojuist hebben gedaan, maar dan zal blijken dat sommig e-mails ander gecodeerd zijn en we niet alle bestanden kunnen openen. In plaats daarvan laten we Python het werk voor ons opknappen door de `email` library te gebruiken.

In [ ]:
import email
import email.policy

def load_email(is_spam, filename, spam_path=SPAM_PATH):
    directory = "spam" if is_spam else "easy_ham"
    with open(os.path.join(spam_path, directory, filename), "rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

We maken met behulp van de functie `load_email` nu een lijst met ham emails en een lijst met spam emails.

In [ ]:
ham_emails = [load_email(is_spam=False, filename=name) for name in ham_filenames]
spam_emails = [load_email(is_spam=True, filename=name) for name in spam_filenames]

Druk ter controle een ham email en een spam email af. Gebruik hiervoor de methode 'as_string()' van het 'email' object.

## Maak de data geschikt voor Machine Learning

Nu hebben we onze data in de vorm van lijsten met e-mails. Om machine learning toe te kunnen passen hebben we echter getallen nodig. In de volgende stappen passen we de data zo aan dat deze bruikbaar wordt voor machine learning.

We werken dit eerst uit op een enkel bestand om het daarna in een functie op alle emails toe te passen.

We maken alleen gebruik van de tekst van de e-mails (hoewel er in de headers vast ook allerlei nuttige informatie staat om te bepalen of het om spam gaat).

In [ ]:
body = spam_emails[13].get_content().strip()
print(body[:1000])

Sommige e-mails bevatten HTML code. Om deze te verwijderen maken we gebruik van een library met de naam `Beautiful Soup`.

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(body, 'html.parser')

In [ ]:
body = soup.get_text()
print(body)

Spam bevat vaker html dan andere e-mails, dus we willen wel weten of een e-mail html bevat. We tellen we het aantal html-tags met behulp van `Beautiful Soup`.

In [ ]:
nhtml = len(soup.find_all())

Om dezelfde reden zijn het aantal links naar websites ook interessant (een link heeft html code `a`). Tel ook deze met `Beautiful Soup` en sla het resultaat op in een variable `nlinks`:

In [ ]:
nlinks = 

Voeg nu voor elke keer dat een htmltag voorkomt het woord `" htmltag "` aan de tekst van de e-mail toe en voeg voor elke keer dat een link voorkomt het woord `" htmllink "` toe. Op deze manier coderen we deze informatie in de tekst zelf.

In [ ]:
body = body + nhtml*" htmltag " + nlinks*" linktag "

Zet nu alle tekst om naar kleine letters (lowercase).

In [ ]:
body = body.lower()
print(body)

De tekst bevat nu nog onnodige lege regels. Deze kunnen we verwijderen met een zogenaamde reguliere expressie. Zoek uit wat de volgende code doet.

In [ ]:
import re

body = re.sub(r'(\s*\n)+', '\n', body)

print(body)

Maak nu zelf onderstaande regular expression af, zodat deze alle opeenvolgende spaties en tabs vervangt door een enkele spatie.

In [ ]:
body = re.sub(r" ", " ", body)

print(body)

Nummers en getallen in tekst zijn lastig omdat er heel veel verschillende van zijn. Het is maar de vraag of bijvoorbeeld een datum iets zegt over spam. We gaan daarom alle getallen vervangen door het woord `" NUMMMER "`.

In [ ]:
body = re.sub(r"\b[\d.]+\b", " NUMMER ", body)

print(body)

We kunnen met behulp van reguliere expressies ook leestekens verwijderen en e-mailadressen en namen van websites verwijderen. We voegen dit alles samen in een functie:

In [ ]:
def clean_email(text):
    #vervang urls door 'httpadr'
    text = re.sub(r"(http|https)://[^\s]*", 'httpaddr', text)
    
    #vervang emailadressen door 'emailadr'
    text = re.sub(r"\b[^\s]+@[^\s]+[.][^\s]+\b", 'emailadr', text)
    
    # verwijder alle leestekens
    text = re.sub(r"([^\w\s]+)|([_-]+)", " ", text)
    
    # vervang alle enters door ' newline '
    text = re.sub(r"\n", " newline ", text)
    
    # vervang opeenvolgende spaties en tabs door een enkele spatie
    text = re.sub(r"\s+", " ", text)
    
    #vervang getallen door 'NUMMER'
    text = re.sub(r"\b[\d.]+\b", " NUMMER ", text)
    
    # verwijder onnodige spaties aan begin en eind
    text = text.strip(" ")
    
    return text

body = clean_email(body)
print (body)

Het resultaat van al onze inspanningen is een lijst met alleen maar woorden gescheiden door spaties.

Maak nu een functie `email_to_text()` die als argument een email krijgt en als resultaat de lijst met woorden in de email teruggeeft.

Deze functie combineert dus bovenstaande stappen: haal met `Beautiful Soup` de tekst uit de e-mail, tel het aantal html tags en links en voegt hier speciale woorden voor toe. Zet vervolgens alle tekst om naar kleine letter en verwijder getallen en leestekens met behulp van `clean_email()`.

Zet deze stappen tussen `try:` en `except:`. Dit deel van de code is al gegeven en zorgt ervoor dat emails die fouten veroorzaken (omdat ze onleesbare tekens of bijvoorbeeld attachments bevatten) worden overgeslagen.

In [ ]:
def email_to_text(email):
    try: 
        
        # voeg hier de stappen toe om een lijst met woorden uit een e-mail te halen.
        
        return body
    except: # handel onleesbare e-mails (i.v.m. attachment) af
        return ''

Met behulp van deze functie kunnen we nu onze data in het juist formaat brengen om een classifier te trainen.
Maak een vector X met alle emails en een target vector y met de bijbehorende labels. Kies bijvoorbeeld 0 als label voor ham en 1 voor spam.

In [ ]:
X = np.array(ham_emails + spam_emails)
y = np.array([0] * len(ham_emails) + [1] * len(spam_emails))

print(len(X))

Splits de data in een training set en een test set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

We willen nu ieder email omzetten naar een vector met getallen. Deze vector bevat een getal voor elk woord dat voorkomt in de dataset. Voor elke email tellen we vervolgens hoe vaak ieder woord voorkomt in die e-mail, dit zijn de features van de e-mails. (Omdat lang niet ieder woord voorkomt in iedere e-mail, zullen heel veel van deze getallen 0 zijn).

We hoeven dit gelukkig niet zelf te doen. sklearn geeft ons de `CountVectorizer` die precies dit doet. Zoek uit hoe deze werkt en pas deze toe op `X_train` om een dataset `X_vec_train` van feature vectoren te krijgen.

`CountVectorizer` heeft als input een lijst met strings van woorden gescheiden door spaties nodig.
Merk ook op dat `CountVectorizer` een optioneel argument preprocessor heeft, dat verwijst naar een functie die de ingevoerde data omzet naar zo'n string. Dit is precies de functie `email_to_text()` die wij zojuist gemaakt hebben.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(preprocessor=email_to_text, min_df = 15)

X_vec_train = vectorizer.fit_transform(X_train)

Onze features bestaan nu dus uit getallen. `CountVectorizer` heeft een functie `get_feature_names()` die op volgorde de woorden teruggeeft die bij deze features horen. Gebruik deze functie om een Pandas `DataFrame` te maken waarmee we de dataset kunnen bekijken. 

In [ ]:
print(len(vectorizer.get_feature_names()))

In [ ]:
pd.DataFrame(X_vec_train.toarray(), columns=vectorizer.get_feature_names())

Zet nu ook de test data in `X_test` om naar features. Let op: hiervoor gebruik je de codering die de vectorizer op basis van `X_train` 'geleerd' heeft. Zoek, als je dat niet al gedaan hebt, in de documentatie het verschil tussen de functies `fit()`, `transform()` en `fit_transform()` op.

In [ ]:
X_vec_test = vectorizer.transform(X_test)

Train en test nu een classifier of deep learning model op basis van deze data.